<a href="https://colab.research.google.com/github/daniloaleixo/generalCrawler/blob/master/generalCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# General Crawler
 

## Configs

In [15]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
import pandas as pd
platform.python_version()

'3.6.9'

In [0]:
# Import scrapy
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import re
import lxml
import urllib.request
import urllib.parse
import http.cookiejar
from lxml.html import fragment_fromstring
import scrapy

## More advanced spider

In [0]:
start_urls = ['http://ekurd.net/all-news']  


In [0]:
TITLE_SELECTOR = 'h2 a ::text'        # This is where the title is stored 
URL_SELECTOR = 'h2 a ::attr(href)'    # This is where the url to the actual article is stored 
DESCRIP_SELECTOR = 'p ::text'         # This is where the article description is stored 
DATE_SELECTOR = 'aside a time::text'  # This is where the publication date is stored

In [0]:
SET_SELECTOR = '.entry-list'          # This is the article class all the other tags fall under


In [7]:
! touch data/test.csv

mkdir: cannot create directory ‘data’: File exists


In [0]:
file = open('data/test.csv', 'w')

In [0]:
class BrickSetSpider(scrapy.Spider):

    name = 'ekurd_spider'
    start_urls = ['http://ekurd.net/all-news']

    # Here we can initialize the script
    def __init__(self):
    
        # Create an empty Pandas dataframe, and an empty list per variable
        self.out = pd.DataFrame(columns=['date', 'title', 'description', 'url'])
        self.dates = []
        self.titles = []
        self.descrips = []
        self.urls = []


    def parse(self, response):

        SET_SELECTOR = '.entry-list'
        
        for news in response.css(SET_SELECTOR):

            DATE_SELECTOR = 'aside a time::text'
            TITLE_SELECTOR = 'h2 a ::text'
            DESCRIP_SELECTOR = 'p ::text'
            URL_SELECTOR = 'h2 a ::attr(href)'

            # We extract the data from each variable and save it in the corresponding list
            self.dates.append(news.css(DATE_SELECTOR).extract())
            self.titles.append(news.css(TITLE_SELECTOR).extract())
            self.descrips.append(news.css(DESCRIP_SELECTOR).extract_first())
            self.urls.append(news.css(URL_SELECTOR).extract())
        
        # Once the lists are complete (the entire page has been scraped) we save it to
        # a temporary dataframe (out2) that will be later concatenated with our main dataframe (out)
        
        # Create the temporary dataframe
        out2 = pd.DataFrame(columns=['date', 'title', 'description', 'url'])
        
        # Save each list to the dataframe
        out2['date'] = self.dates
        out2['title'] = self.titles
        out2['description'] = self.descrips
        out2['url'] = self.urls
        
        # Ensure type is string (useful to bypass any non properly formatted date time and other possible errors)
        for c in out2.columns:
            out2[c] = out2[c].astype(str)
        
        # Merge it with our main dataframe (out)
        self.out = pd.concat([self.out, out2], axis=0)
        
        # Save it to file
        self.out.to_csv('data/test.csv')

In [10]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(BrickSetSpider)
process.start()

2019-12-09 00:18:20 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2019-12-09 00:18:20 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Nov  7 2019, 10:44:02) - [GCC 8.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2019-12-09 00:18:20 [scrapy.crawler] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-12-09 00:18:20 [scrapy.extensions.telnet] INFO: Telnet Password: 8482b8b29826d64b
2019-12-09 00:18:20 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-12-09 00:18:21 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddlew

2019-12-09 00:18:21 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://ekurd.net/all-news> from <GET http://ekurd.net/all-news>
2019-12-09 00:18:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ekurd.net/all-news> (referer: None)
2019-12-09 00:18:21 [scrapy.core.engine] INFO: Closing spider (finished)
2019-12-09 00:18:21 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 526,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 23793,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 1,
 'downloader/response_status_count/301': 1,
 'elapsed_time_seconds': 0.771634,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2019, 12, 9, 0, 18, 21, 789754),
 'log_count/DEBUG': 2,
 'log_count/INFO': 10,
 'memusage/max': 180555776,
 'memusage/startup': 180555776,
 'response_received_count': 1,
 'scheduler/dequeued': 2,
 'scheduler/dequeued/me

In [11]:
!ls

data  quoteresult.jl  quoteresult.json	sample_data


In [12]:
! tail data/test.csv

4,"['December 6, 2019']","['Sistani says no foreign meddling to choose new Iraqi PM, in nod to Iran']","
                        BAGHDAD,— Iraq’s top Shi’ite Muslim cleric said on Friday that a new prime minister must be chosen without foreign interference in an apparent nod to …                    ",['https://ekurd.net/sistani-says-no-foreign-meddling-2019-12-06']
5,"['December 6, 2019']","['Gas explosion in Iranian Kurdistan kills at least 11, injures dozens']","
                        SEQIZ, Iranian Kurdistan,— At least 11 people were killed and dozens were injured in a gas explosion in Iran’s western Kurdistan province (Iranian …                    ",['https://ekurd.net/gas-explosion-iranian-kurdistan-2019-12-06']
6,"['December 6, 2019']",['Iraqi parliament passes law reforming electoral IHEC commission'],"
                        BAGHDAD,— Iraq’s parliament on Thursday passed the law of the Independent High Electoral Commission (IHEC), as part of efforts by the Iraqi government …

In [0]:
test = pd.read_csv("data/test.csv")

In [14]:
test

,Unnamed: 0,date,title,description,url
0,0,"['December 8, 2019']",['Masrour Barzani outlines Kurdistan cabinet’s...,"\r\n HEWLÊR-Erbil, Iraq...",['https://ekurd.net/masrour-barzani-outlines-p...
1,1,"['December 8, 2019']",['Drone hits Iraqi cleric Muqtada Sadr’s home ...,"\r\n BAGHDAD,— An armed...",['https://ekurd.net/drone-hits-iraqi-cleric-mu...
2,2,"['December 7, 2019']",['Citizenship and Identity – Part V'],\r\n Dr. Sorran Arasan ...,['https://ekurd.net/citizenship-and-identity-p...
3,3,"['December 7, 2019']",['4 Challenges of Being an Insurance Agent and...,"\r\n As with any job, b...",['https://ekurd.net/challenges-insurance-agent...
4,4,"['December 6, 2019']",['Sistani says no foreign meddling to choose n...,"\r\n BAGHDAD,— Iraq’s t...",['https://ekurd.net/sistani-says-no-foreign-me...
5,5,"['December 6, 2019']",['Gas explosion in Iranian Kurdistan kills at ...,"\r\n SEQIZ, Iranian Kur...",['https://ekurd.net/gas-explosion-iranian-kurd...
6,6,"['December 6, 2019']",['Iraqi parliament passes law reforming electo...,"\r\n BAGHDAD,— Iraq’s p...",['https://ekurd.net/iraqi-parliament-passes-la...
7,7,"['December 6, 2019']",['Business Solutions That Makes Management a W...,\r\n On a day-to-day ba...,['https://ekurd.net/business-solutions-managem...
8,8,"['December 6, 2019']",['Importance of Digital Invoice To Your Busine...,\r\n Introducing techno...,['https://ekurd.net/importance-digital-invoice...
